<a href="https://colab.research.google.com/github/zulfanHamka/technicaltestDE_web_berita/blob/main/Standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
from datetime import datetime
from zoneinfo import ZoneInfo
import os
import sys

# Konfigurasi skrip
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
}
INTERVAL_DETIK = 300
FILE_LINKS_TERLIHAT = 'seen_links.txt'
WIB = ZoneInfo("Asia/Jakarta")

# Fungsi ambil isi artikel
def get_article_content(link):
    try:
        res = requests.get(link, headers=HEADERS)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, 'lxml')
        div_content = soup.find('article', class_='detailsContent')
        if not div_content:
            return "Konten tidak ditemukan"
        paragraphs = div_content.find_all('p')
        return ' '.join([p.get_text(strip=True) for p in paragraphs])
    except requests.exceptions.RequestException as e:
        print(f"Error saat mengambil artikel: {link} | {e}")
        return ''

# Fungsi muat link yang sudah dilihat
def muat_links_terlihat():
    if not os.path.exists(FILE_LINKS_TERLIHAT):
        return set()
    with open(FILE_LINKS_TERLIHAT, 'r', encoding='utf-8') as f:
        return set(line.strip() for line in f)

# Fungsi simpan link baru
def simpan_link_baru(link):
    with open(FILE_LINKS_TERLIHAT, 'a', encoding='utf-8') as f:
        f.write(link + '\n')

# Fungsi utama
def main():
    links_terlihat = muat_links_terlihat()
    print(f"Memuat {len(links_terlihat)} link yang sudah diproses.")
    print(f"Memulai monitoring artikel baru (Interval: {INTERVAL_DETIK} detik)")

    # Loop utama
    while True:
        waktu_sekarang_wib = datetime.now(WIB)
        print(f"\n[{waktu_sekarang_wib.strftime('%Y-%m-%d %H:%M:%S')}] Memeriksa artikel baru...")

        url = 'https://www.bisnis.com/terbaru'
        artikel_baru_ditemukan = []

        try:
            res = requests.get(url, headers=HEADERS)
            soup = BeautifulSoup(res.text, 'lxml')
            articles = soup.find_all('div', class_='artItem')

            for art in articles:
                a_tag = art.find('a')
                if not (a_tag and 'href' in a_tag.attrs):
                    continue

                link = a_tag['href']

                # Cek jika link baru
                if link not in links_terlihat:
                    print(f"Artikel baru ditemukan: {link}")

                    links_terlihat.add(link)
                    simpan_link_baru(link)

                    h4_tag = art.find('h4')
                    title = h4_tag.get_text(strip=True) if h4_tag else "Judul tidak ditemukan"
                    content = get_article_content(link)

                    try:
                        date_part = link.split('/')[4]
                        time_info = f"{date_part[:4]}-{date_part[4:6]}-{date_part[6:]}"
                    except IndexError:
                        time_info = waktu_sekarang_wib.strftime('%Y-%m-%d')

                    # Kumpulkan data artikel baru
                    artikel_baru_ditemukan.append({
                        'Title': title,
                        'Link': link,
                        'Time': time_info,
                        'Content': content,
                        'ScrapedAt': datetime.now(WIB).isoformat()
                    })
                    time.sleep(1) # Jeda sopan

            # Simpan ke file jika ada artikel baru
            if artikel_baru_ditemukan:
                timestamp = datetime.now(WIB).strftime('%Y-%m-%d_%H%M%S')
                file_name = f'artikel_baru_{timestamp}.json'
                with open(file_name, 'w', encoding='utf-8') as f:
                    json.dump(artikel_baru_ditemukan, f, ensure_ascii=False, indent=4)
                print(f"Menyimpan {len(artikel_baru_ditemukan)} artikel baru ke file {file_name}")
            else:
                print("Tidak ada artikel baru.")

        except requests.exceptions.RequestException as e:
            print(f"Gagal mengakses situs: {e}")

        # Jeda sebelum pemeriksaan berikutnya
        print(f"Menunggu {INTERVAL_DETIK} detik...")
        time.sleep(INTERVAL_DETIK)

# Cek versi python sebelum menjalankan
if __name__ == "__main__":
    if sys.version_info < (3, 9):
        print("Error: Skrip ini memerlukan Python 3.9+ untuk fitur zona waktu.")
        sys.exit(1)
    main()

Memuat 0 link yang sudah diproses.
Memulai monitoring artikel baru (Interval: 300 detik)

[2025-09-25 13:05:30] Memeriksa artikel baru...
Tidak ada artikel baru.
Menunggu 300 detik...

[2025-09-25 13:10:31] Memeriksa artikel baru...
Tidak ada artikel baru.
Menunggu 300 detik...


KeyboardInterrupt: 

### tidak ada file json karena artikel tidak ditemukan